# Análise de Tópicos

#### Importação de Bibliotecas

In [1]:
import pandas as pd
import spacy
from bertopic import BERTopic

from collections import Counter

import re
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
from gensim.utils import simple_preprocess

import nltk
nltk.download(["stopwords", "rslp"])
stopwords = nltk.corpus.stopwords.words("portuguese")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [3]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForSequenceClassification  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModelForTokenClassification
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline
import torch

In [4]:
# from openai import OpenAI

#### Leitura do arquivo

O arquivo com os dados da avaliação do restaurante Guacamole, retirados da plataforma TripAdvisor em https://www.tripadvisor.com.br/Restaurant_Review-g303506-d3399400-Reviews-or2000-Guacamole_Cocina_Mexicana_Barra_da_Tijuca-Rio_de_Janeiro_State_of_Rio_de_J.html possui os comentários dos usuários em relação às suas respectivas avaliações do restaurante e a data de tal avaliação.

In [5]:
df_reviews = pd.read_csv("guacamole_reviews.csv")
df_reviews.head()

,Date,Review
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já..."
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest..."
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó..."


In [6]:
df_comments = df_reviews["Review"]
df_dates = df_reviews["Date"]

#### Limpeza dos dados

As funções em seguir estão relacionadas ao pré-processamento de texto em linguagem natural utilizando o português. 
- A função ```remove_stopwords``` realiza a remoção de palavras de parada (stopwords) de textos em português. 
- A função ```lemmatizer``` realiza a lematização de um texto em português, preservando apenas certas classes gramaticais.
- A função ```parse_date``` realiza a conversão de uma data no formato "(dia) de (mês) de (ano)" para "YYYY-MM-DD".

In [7]:
def remove_stopwords(text):
    words = simple_preprocess(text)
    phrase_adjusted = " ".join([word for word in words if word not in stopwords])
    return phrase_adjusted.lower()

spacy_lemma = spacy.load("pt_core_news_sm")

def lemmatizer(text, postags_permit=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    doc = spacy_lemma(text.lower())
    doc_lemma = " ".join([token.lemma_ for token in doc if token.pos_ in postags_permit])
    return doc_lemma

In [8]:
def parse_date(date_str):
    months = {
        'janeiro': 'January', 'fevereiro': 'February', 'março': 'March', 'abril': 'April',
        'maio': 'May', 'junho': 'June', 'julho': 'July', 'agosto': 'August',
        'setembro': 'September', 'outubro': 'October', 'novembro': 'November', 'dezembro': 'December'
    }
    match = re.match(r'(\d+) de (\w+) de (\d+)', date_str)
    if match:
        day, month, year = match.groups()
        month = months[month.lower()]
        return datetime(int(year), list(months.values()).index(month) + 1, int(day)).strftime("%Y-%m-%d")
    else:
        return None

Aplicamos as mudanças nos comentários das avaliações e adicionamos uma nova coluna no DataFrame com os resultados obtidos.

In [9]:
df_reviews["Review Lemma"] = df_reviews["Review"].map(remove_stopwords)
df_reviews["Review Lemma"] = df_reviews["Review"].map(lemmatizer)

In [10]:
df_reviews["Date Formatted"] = df_reviews["Date"].apply(parse_date)

In [11]:
df_reviews.head()

,Date,Review,Review Lemma,Date Formatted
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27


In [12]:
df_comments = df_reviews["Review Lemma"]
doc_comments = df_comments.to_list()
doc_comments[:5]

['atendimento maravilhoso matheu muito atencioso colocar playlist atendente super simpatico',
 'gostar muito prato atendimento solicito comida saboroso certeza voltar mais vez',
 'ameir bom cozinha mexicano já comi espaço muito animar imersivo decoração casa linda atendimento excelente sempre',
 'bom ambiente decoração funcionário prestativo comida boa música agradável divertido mariachi',
 'amoma atendimento matheu bom amar atendimento sensacional comida muito saboroso']

#### BERTopic

Utilizando os parâmetros padronizados do BERTopic, obteremos os tópicos, do mais frequente ao menos frequente. 
- Se desejamos visualizar a frequência de cada tópico, podemos utilizar ```get_topic_freq()``` para obter os valores.
- Se desejamos visualizar as informações de todos os tópicos obtidos, podemos utilizar ```get_topic_info()``` para obter o número de documentos que estão relacionados a cada tópico e o nome dele, além de suas representações textuais.
- Se desejamos visualizar as informações de um tópico em específico, podemos utilizar o ```get_topic()``` para obter as palavras que pertencem ao tópico e os seus _scores_ de c-TF-IDF, ou seja, uma representação numérica do quão frequente e única aquela palavra é para o documento.

Vale lembrar que o tópico de número **-1** refere-se a todos os valores discrepantes e normalmente deve ser ignorado.

In [13]:
topic_model = BERTopic(language="portuguese", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(doc_comments)

2024-01-10 15:02:49,477 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/90 [00:00<?, ?it/s]

2024-01-10 15:04:07,501 - BERTopic - Embedding - Completed ✓
2024-01-10 15:04:07,501 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-10 15:04:50,340 - BERTopic - Dimensionality - Completed ✓
2024-01-10 15:04:50,348 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-10 15:04:50,793 - BERTopic - Cluster - Completed ✓
2024-01-10 15:04:50,816 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-10 15:04:51,068 - BERTopic - Representation - Completed ✓


In [14]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,194,-1_muito_bom_bem_lugar,"[muito, bom, bem, lugar, mais, animação, comid...",[comida maravilhoso decoração incrível funcion...
1,0,736,0_mexicano_guacamole_comida_restaurante,"[mexicano, guacamole, comida, restaurante, mai...",[restaurante ter comida mexicano muito gostoso...
2,1,655,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[experiência incrível comer ótimo lugar super ...
3,2,352,2_música_bom_muito_ambiente,"[música, bom, muito, ambiente, atendimento, mu...",[ambiente muito agradável música bom excelente...
4,3,234,3_aniversário_comemorar_amigo_muito,"[aniversário, comemorar, amigo, muito, bom, lu...",[bom local comemorar aniversário decoração mui...


In [21]:
topic_model.get_topic_freq()

,Topic,Count
2,0,736
1,1,655
3,2,352
8,3,234
4,4,222
0,-1,194
7,5,144
11,6,105
5,7,77
13,8,41


In [17]:
topic_model.get_topic(0)

[('mexicano', 0.0745800914077885),
 ('guacamole', 0.03984880146390574),
 ('comida', 0.0361451742865957),
 ('restaurante', 0.035575550845103616),
 ('mais', 0.03430381511151614),
 ('bom', 0.03342608512583137),
 ('muito', 0.03021335857663606),
 ('ter', 0.02690972280962471),
 ('bem', 0.02570735690089816),
 ('ambiente', 0.02515709419607347)]

#### Visualização dos tópicos

In [18]:
topic_model.visualize_barchart(top_n_topics=10)

In [33]:
topic_model.visualize_hierarchy()

In [34]:
dates_reviews = df_reviews["Date Formatted"].to_list()

In [ ]:
# topics_over_time = topic_model.topics_over_time(doc_comments, dates_reviews, datetime_format="%Y-%m-%d", nr_bins=20)

In [ ]:
# topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

#### DataFrame com informações completas

In [35]:
df_topics = df_reviews.copy()

df_topics["Topics"] = topics

topic_name = freq.drop(columns=["Count"]).rename(columns={"Topic": "Topics", "Name": "Names"})
df_topics = df_topics.merge(topic_name, how="left")

df_topics.head()

,Date,Review,Review Lemma,Date Formatted,Topics,Names,Representation,Representative_Docs
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04,-1,-1_muito_bom_bem_lugar,"[muito, bom, bem, lugar, mais, animação, comid...",[comida maravilhoso decoração incrível funcion...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[experiência incrível comer ótimo lugar super ...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01,0,0_mexicano_guacamole_comida_restaurante,"[mexicano, guacamole, comida, restaurante, mai...",[restaurante ter comida mexicano muito gostoso...
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29,2,2_música_bom_muito_ambiente,"[música, bom, muito, ambiente, atendimento, mu...",[ambiente muito agradável música bom excelente...
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[experiência incrível comer ótimo lugar super ...


# Análise de Sentimentos

In [36]:
model = AutoModelForSequenceClassification.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student')
tokenizer = AutoTokenizer.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student', do_lower_case=False)
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [37]:
sentiment_task("Eu sou feliz")

[{'label': 'positive', 'score': 0.9528123140335083}]

#### Classificação das avaliações e salvamento dos resultados no DataFrame

In [38]:
%%time

df_topics["Sentiment"] = df_topics["Review"].apply(lambda x: sentiment_task(x)[0]["label"])
df_topics["Sentiment Score"] = df_topics["Review"].apply(lambda x: sentiment_task(x)[0]["score"])    

CPU times: total: 29min 55s
Wall time: 4min 52s


In [39]:
df_topics.head()

,Date,Review,Review Lemma,Date Formatted,Topics,Names,Representation,Representative_Docs,Sentiment,Sentiment Score
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04,-1,-1_muito_bom_bem_lugar,"[muito, bom, bem, lugar, mais, animação, comid...",[comida maravilhoso decoração incrível funcion...,positive,0.989543
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[experiência incrível comer ótimo lugar super ...,positive,0.626670
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01,0,0_mexicano_guacamole_comida_restaurante,"[mexicano, guacamole, comida, restaurante, mai...",[restaurante ter comida mexicano muito gostoso...,positive,0.967855
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29,2,2_música_bom_muito_ambiente,"[música, bom, muito, ambiente, atendimento, mu...",[ambiente muito agradável música bom excelente...,positive,0.940317
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[experiência incrível comer ótimo lugar super ...,positive,0.979101


In [40]:
df_topics.to_csv('guacamole_topics_sentiment.csv', index=False)

In [41]:
count_sentiment = df_topics["Sentiment"].value_counts()
count_sentiment

Sentiment
positive    2648
negative     218
neutral        6
Name: count, dtype: int64

Pelo valor do "Sentiment Score" calculamos uma média para sabermos qual que foi o "Score" médio de cada um dos tópicos.

In [57]:
df_grouped = df_topics.groupby('Topics')

df_topics_avg_sentiment = df_grouped['Sentiment Score'].mean().reset_index()

df_topics_avg_sentiment.head(10)

,Topics,Sentiment Score
0,-1,0.824361
1,0,0.843911
2,1,0.884396
3,2,0.898811
4,3,0.864975
5,4,0.898045
6,5,0.885367
7,6,0.856033
8,7,0.670230
9,8,0.812739


# Extração de Entidade Nomeada

In [ ]:
model_ner = AutoModelForTokenClassification.from_pretrained('51la5/roberta-large-NER')
tokenizer_ner = AutoTokenizer.from_pretrained('51la5/roberta-large-NER', do_lower_case=False)
ner_task = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner)

In [ ]:
ner_task("Julia não gosta de Londres nem Berlim")

In [ ]:
def classify_ner(text):
    """
    Token classification function using a pretrained model.

    Parameters:
    - text: Input text to be tokenized and classified.

    Returns:
    List of tuples containing predicted pairs (token, label) for the input text.

    Example:
    Input:  "Julia is tired of living in London."
    Output: [('Julia', 'B-PESSOA'), ('is', 'O'), ('tired', 'O'), ('of', 'O'),
            ('living', 'O'), ('in', 'O'), ('London', 'B-LOCAL'), ('.', 'O')]
    """

    inputs = tokenizer_ner(text, max_length=512, truncation=True, return_tensors="pt")
    tokens = inputs.tokens()

    outputs = model_ner(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)

    results = []

    for token, prediction in zip(tokens, predictions[0].numpy()):
        label = model_ner.config.id2label.get(prediction, 'O')
        if label != 'O':
            results.append((token, label))

    return results

In [ ]:
df_topics["Token Predictions"] = df_topics["Review"].apply(classify_ner)

In [ ]:
df_topics["Token Predictions"]

In [ ]:
def merge_tokens(token_predictions):
    """
    Function to merge consecutive tokens that start with "_" and have the same label.

    Parameters:
    - token_predictions: List of tuples containing predicted pairs (token, label).

    Returns:
    List of merged tuples where consecutive tokens starting with "_"
    and having the same label are combined.

    Example:
    Input:  [('▁Mathe', 'I-PER'), ('us', 'I-PER')]
    Output: [('▁Matheus', 'I-PER')]
    """

    merged_results = []
    current_token = ""
    current_label = ""

    for token, label in token_predictions:
        if token.startswith("▁"):
            if current_token:
                merged_results.append((current_token, current_label))
            current_token = token[1:]
            current_label = label
        else:
            current_token += token
            current_label = label

    if current_token:
        merged_results.append((current_token, current_label))

    return merged_results

In [ ]:
df_topics["Token Predictions Corrected"] = df_topics["Token Predictions"].apply(merge_tokens)

In [ ]:
df_topics["Token Predictions Corrected"]

In [ ]:
token_predictions_corrected = df_topics["Token Predictions Corrected"]

flat_list = [item for sublist in token_predictions_corrected for item in sublist]

counter = Counter(flat_list)

most_common_elements = counter.most_common(10)

most_common_elements